https://langchain-ai.github.io/langgraph/tutorials/reflection/reflection/

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
from langchain_openai import ChatOpenAI


In [2]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatOpenAI(model="gpt-4o-mini")

generate = prompt | llm

In [3]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

**The Relevance of "The Little Prince" in Modern Childhood**

Antoine de Saint-Exupéry's timeless tale, "The Little Prince," continues to resonate with readers of all ages, particularly children, even decades after its publication. Its themes of innocence, exploration, and the importance of relationships are not only universal but also crucial in today's fast-paced, technology-driven world. The relevance of "The Little Prince" in modern childhood lies in its ability to nurture imagination, encourage emotional intelligence, and foster meaningful connections in an age often overshadowed by superficiality.

First and foremost, "The Little Prince" serves as a powerful catalyst for imagination and creativity. In an era dominated by screens and digital content, children are often inundated with pre-packaged narratives that leave little room for personal interpretation or creative thought. The whimsical journey of the Little Prince, with his encounters on various planets and the lessons he le

In [4]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [5]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

**Essay Critique and Recommendations**

Your essay on the relevance of "The Little Prince" in modern childhood presents a thoughtful exploration of its themes and their significance in today’s world. You’ve effectively highlighted key aspects of the narrative that resonate with children, such as imagination, emotional intelligence, and the critique of superficial values. Here are some critiques and recommendations to enhance your essay further:

### Strengths:
1. **Clarity of Argument**: Your thesis is clearly articulated, and you provide a structured argument that supports your main points. Each paragraph transitions smoothly into the next.
2. **Thematic Depth**: You’ve successfully identified and explained the major themes of the book, linking them to contemporary issues faced by children.
3. **Engagement with Source Material**: References to specific aspects of the narrative (e.g., the relationship between the Little Prince and the Fox) strengthen your analysis and show a good under

In [6]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

Thank you for your insightful critique and recommendations! I appreciate your feedback, and I will take your suggestions into account to enhance the essay. Below is a revised version that incorporates more depth of analysis, addresses potential counterarguments, and varies sentence structure for better engagement.

---

**The Relevance of "The Little Prince" in Modern Childhood**

Antoine de Saint-Exupéry's timeless tale, "The Little Prince," continues to resonate with readers of all ages, particularly children, even decades after its publication. Its themes of innocence, exploration, and the importance of relationships are not only universal but also crucial in today's fast-paced, technology-driven world. The relevance of "The Little Prince" in modern childhood lies in its ability to nurture imagination, encourage emotional intelligence, and foster meaningful connections in an age often overshadowed by superficiality.

First and foremost, "The Little Prince" serves as a powerful catal

In [11]:
from typing import List, Sequence

from langgraph.graph import END, MessageGraph, START


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")


def should_continue(state: List[BaseMessage]):
    if len(state) > 2:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [12]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Generate an essay on the topicality of The Little Prince and its message in modern life"
        )
    ],
):
    print(event)
    print("---")

{'generate': AIMessage(content='**The Timeless Relevance of "The Little Prince" in Modern Life**\n\nAntoine de Saint-Exupéry\'s beloved novella, "The Little Prince," first published in 1943, continues to resonate with readers across generations and cultures. Its enchanting narrative and profound philosophical insights delve into themes of love, loss, and the essence of human relationships. In a world increasingly dominated by materialism, technology, and a disconnect from the natural world, the messages embedded in "The Little Prince" are more relevant than ever. This essay will explore the novella\'s key themes, its critique of adult behavior, and its call for deeper emotional connections, ultimately illustrating why its message remains vital in contemporary society.\n\nAt the heart of "The Little Prince" lies a poignant critique of adult behavior and societal values. The story contrasts the innocent perspective of children with the often rigid and narrow-minded views of adults. The L

In [9]:
print(event)

{'generate': AIMessage(content='**The Timeless Relevance of "The Little Prince" in Modern Life**\n\nAntoine de Saint-Exupéry’s "The Little Prince," first published in 1943, has transcended its original context to become a profound exploration of human nature, relationships, and the essence of life itself. This enchanting tale, often categorized as a children\'s book, resonates deeply with adult readers due to its multilayered themes and philosophical insights. In modern life, where complexity often overshadows simplicity, "The Little Prince" serves as a compelling reminder of the values of love, friendship, and the importance of seeing beyond the surface.\n\nAt the heart of "The Little Prince" is the idea that what is essential is often invisible to the eye. This poignant message speaks volumes in today\'s fast-paced world, where materialism and superficiality seem to dominate. The Little Prince’s journey from his asteroid to Earth symbolizes a quest for deeper understanding and connec

In [10]:
ChatPromptTemplate.from_messages(event[END]).pretty_print()

KeyError: '__end__'